In [16]:
import ivy

# Write Ivy Code

## Data Structures

In [2]:
%%capture
ivy.set_backend("torch")

In [3]:
x = ivy.array([1, 2, 3])
print(type(x))

<class 'ivy.data_classes.array.array.Array'>


In [4]:
x = ivy.native_array([1, 2, 3])
print(type(x))

<class 'torch.Tensor'>


## Ivy Functional API

In [10]:
ivy.set_backend("jax")
x1, x2 = ivy.array([[1], [2], [3]]), ivy.array([[1, 2, 3]])
output = ivy.matmul(x1, x2)
print(type(output.to_native()))

<class 'jaxlib.xla_extension.ArrayImpl'>


In [11]:
ivy.set_backend("tensorflow")
x1, x2 = ivy.array([[1], [2], [3]]), ivy.array([[1, 2, 3]])
output = ivy.matmul(x1, x2)
print(type(output.to_native()))

<class 'tensorflow.python.framework.ops.EagerTensor'>


In [12]:
ivy.set_backend("torch")
x1, x2 = ivy.array([[1], [2], [3]]), ivy.array([[1, 2, 3]])
output = ivy.matmul(x1, x2)
print(type(output.to_native()))

<class 'torch.Tensor'>


In [13]:
ivy.set_backend("numpy")
x1, x2 = ivy.array([[1], [2], [3]]), ivy.array([[1, 2, 3]])
output = ivy.matmul(x1, x2)
print(type(output.to_native()))

<class 'numpy.ndarray'>


In [15]:
ivy.set_array_mode(False)

ivy.set_backend("jax")
x1, x2 = ivy.native_array([[1], [2], [3]]), ivy.native_array([[1, 2, 3]])
output = ivy.matmul(x1, x2)
print(type(output))

ivy.set_backend("tensorflow")
x1, x2 = ivy.native_array([[1], [2], [3]]), ivy.native_array([[1, 2, 3]])
output = ivy.matmul(x1, x2)
print(type(output))

ivy.set_backend("torch")
x1, x2 = ivy.native_array([[1], [2], [3]]), ivy.native_array([[1, 2, 3]])
output = ivy.matmul(x1, x2)
print(type(output))

ivy.set_array_mode(True)

<class 'jaxlib.xla_extension.ArrayImpl'>
<class 'ivy.data_classes.array.array.Array'>
<class 'torch.Tensor'>


# Unify Code

In [1]:
import ivy
import torch

In [8]:
def normalize(x):
    mean = torch.mean(x)
    std = torch.std(x)
    return torch.div(torch.sub(x, mean), std)

In [9]:
normalize = ivy.unify(normalize, source="torch")

In [10]:
# import the frameworks
import numpy as np
import jax.numpy as jnp
import tensorflow as tf

In [11]:
x = np.random.uniform(size=10).astype(np.float32)
ivy.set_backend("numpy")
print(normalize(x))

TypeError: mean(): argument 'input' (position 1) must be Tensor, not NewNDArray

In [14]:
# jax
x_ = jnp.array(x)
ivy.set_backend("jax")
print(normalize(x_))

ivy.array([ 0.97889817, -0.31358162, -0.1838607 ,  0.68678105,  0.4830147 ,
       -1.90820944,  1.37414706, -1.01904213, -0.6111415 ,  0.51299494])


In [16]:
# tensorflow
x_ = tf.constant(x)
ivy.set_backend("tensorflow")
print(normalize(x_))

ivy.array([ 0.97889823, -0.31358165, -0.18386073,  0.68678117,  0.48301476,
       -1.90820968,  1.3741473 , -1.01904213, -0.61114156,  0.512995  ])


In [17]:
# torch
x_ = torch.tensor(x)
ivy.set_backend("torch")
print(normalize(x_))

ivy.array([ 0.97889805, -0.31358185, -0.18386094,  0.68678093,  0.48301455,
       -1.9082098 ,  1.37414706, -1.01904237, -0.6111418 ,  0.51299483])


# Trace Code

In [18]:
import ivy
import torch

def normalize(x):
    mean = torch.mean(x)
    std = torch.std(x)
    return torch.div(torch.sub(x, mean), std)
    
normalize = ivy.unify(normalize, source="torch")

In [19]:
ivy.set_backend("jax")

import jax

key = jax.random.PRNGKey(42)
x = jax.random.uniform(key, shape=(10,))

In [20]:
normalize(x)

ivy.array([ 0.55563939, -0.65538698, -1.14150512,  1.46951973,  1.30220282,
       -1.14739656, -0.5701794 , -0.91962665,  0.51028997,  0.59644389])

In [21]:
ivy.set_backend("jax")
traced = ivy.trace_graph(normalize)

In [22]:
traced(x)

Array([ 0.5556394 , -0.655387  , -1.1415051 ,  1.4695197 ,  1.3022028 ,
       -1.1473966 , -0.5701794 , -0.91962665,  0.51028997,  0.5964439 ],      dtype=float32)

In [25]:
%%timeit
normalize(x)

24.6 ms ± 3.46 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [26]:
%%timeit
traced(x)

695 µs ± 3.67 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


# Transpile Code